### Prueba del código con valores de internet del modelo de entrenamiento

In [7]:
import numpy as np
import pandas as pd
import networkx as nx
from matplotlib import pyplot as plt
%matplotlib inline
from pprint import pprint

In [8]:
def forward(V, a, b, initial_distribution):
    alpha = np.zeros((V.shape[0], a.shape[0]))
    alpha[0, :] = initial_distribution * b[:, V[0]]

    for t in range(1, V.shape[0]):
        for j in range(a.shape[0]):
            # Matrix Computation Steps
            #                  ((1x2) . (1x2))      *     (1)
            #                        (1)            *     (1)
            alpha[t, j] = alpha[t - 1].dot(a[:, j]) * b[j, V[t]]
            
    return alpha
 

def backward(V, a, b):
    beta = np.zeros((V.shape[0], a.shape[0]))
    
    # setting beta(T) = 1
    beta[V.shape[0] - 1] = np.ones((a.shape[0]))

    # Loop in backward way from T-1 to
    # Due to python indexing the actual loop will be T-2 to 0
    for t in range(V.shape[0] - 2, -1, -1):
        for j in range(a.shape[0]):
            beta[t, j] = (beta[t + 1] * b[:, V[t + 1]]).dot(a[j, :])
    return beta
 

def baum_welch(V, a, b, initial_distribution, n_iter=100):
    M = a.shape[0]
    T = len(V)

    for n in range(n_iter):
        alpha = forward(V, a, b, initial_distribution)
        beta = backward(V, a, b)

        xi = np.zeros((M, M, T - 1))
        for t in range(T - 1):
            denominator = np.dot(np.dot(alpha[t, :].T, a) * b[:, V[t + 1]].T, beta[t + 1, :])
            for i in range(M):
                numerator = alpha[t, i] * a[i, :] * b[:, V[t + 1]].T * beta[t + 1, :].T
                xi[i, :, t] = numerator / denominator
                
        gamma = np.sum(xi, axis=1)
        a = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))

        # Add additional T'th element in gamma
        gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))

        K = b.shape[1]
        denominator = np.sum(gamma, axis=1)
        for l in range(K):
            b[:, l] = np.sum(gamma[:, V == l], axis=1)

        b = np.divide(b, denominator.reshape((-1, 1)))
        
    return {"a":a, "b":b}

data = pd.read_csv('/home/katherine/compu_kathe/9no_semestre/seminario/proyecto_seminario/prueba_internet.csv')
 
V = data['Visible'].values
 
# Transition Probabilities
a = np.ones((2, 2))
a = a / np.sum(a, axis=1)
print('A: ', a) 
# Emission Probabilities
b = np.array(((1, 3, 5), (2, 4, 6)))
#print('B: ', b)
b = b / np.sum(b, axis=1).reshape((-1, 1))
#print('B: ', b)
# Equal Probabilities for the initial distribution
initial_distribution = np.array((0.5, 0.5))
 
print(baum_welch(V, a, b, initial_distribution, n_iter=100))

A:  [[0.5 0.5]
 [0.5 0.5]]
{'a': array([[0.53816345, 0.46183655],
       [0.48664443, 0.51335557]]), 'b': array([[0.16277513, 0.26258073, 0.57464414],
       [0.2514996 , 0.27780971, 0.47069069]])}


In [11]:
def viterbi(pi, a, b, obs):
    
    nStates = np.shape(b)[0]
    T = np.shape(obs)[0]
   
    # Camino inicial. Matrices con ceros.
    path = np.zeros(T)
    # delta --> mayor probabilidad de cualquier camino que alcance el estado i.
    delta = np.zeros((nStates, T))
    # phi --> argmax por paso de tiempo para cada estado
    phi = np.zeros((nStates, T))
    
    # inicio
    delta[:, 0] = pi * b[:, obs[0]]
    phi[:, 0] = 0

    print('\nInicio. Caminar hacia adelante\n')    
    # Extensión del algoritmo directo
    for t in range(1, T):
        for s in range(nStates):
                
            delta[s, t] = np.max(delta[:, t-1] * a[:, s]) * b[s, obs[t]] 
            phi[s, t] = np.argmax(delta[:, t-1] * a[:, s])
            print('s={s} and t={t}: phi[{s}, {t}] = {phi}'.format(s=s, t=t, phi=phi[s, t]))
    
    # Encontrar el camino óptimo
    print('-'*50)
    print('Iniciar retroceso\n')
    path[T-1] = np.argmax(delta[:, T-1])
    
    
    for t in range(T-2, -1, -1):
        
        x = int(path[t+1])
    
        path[t] = phi[x, [t+1]]
        print('path[{}] = {}'.format(t, path[t]))
        
    return path, delta, phi

print(a)
print(b)
print(initial_distribution)
obs_vi = np.array([0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,0,0,0,1])
path, delta, phi = viterbi(initial_distribution, a, b, obs_vi)
print('\n La mejor ruta del estado: \n', path)
print('\n delta:\n', delta)
print('phi:\n', phi)


[[0.5 0.5]
 [0.5 0.5]]
[[ 41.2         67.5        137.89473684]
 [ 61.8         67.5        124.10526316]]
[0.5 0.5]

Inicio. Caminar hacia adelante

s=0 and t=1: phi[0, 1] = 1.0
s=1 and t=1: phi[1, 1] = 1.0
s=0 and t=2: phi[0, 2] = 1.0
s=1 and t=2: phi[1, 2] = 1.0
s=0 and t=3: phi[0, 3] = 1.0
s=1 and t=3: phi[1, 3] = 1.0
s=0 and t=4: phi[0, 4] = 1.0
s=1 and t=4: phi[1, 4] = 1.0
s=0 and t=5: phi[0, 5] = 0.0
s=1 and t=5: phi[1, 5] = 0.0
s=0 and t=6: phi[0, 6] = 0.0
s=1 and t=6: phi[1, 6] = 0.0
s=0 and t=7: phi[0, 7] = 0.0
s=1 and t=7: phi[1, 7] = 0.0
s=0 and t=8: phi[0, 8] = 0.0
s=1 and t=8: phi[1, 8] = 0.0
s=0 and t=9: phi[0, 9] = 0.0
s=1 and t=9: phi[1, 9] = 0.0
s=0 and t=10: phi[0, 10] = 0.0
s=1 and t=10: phi[1, 10] = 0.0
s=0 and t=11: phi[0, 11] = 0.0
s=1 and t=11: phi[1, 11] = 0.0
s=0 and t=12: phi[0, 12] = 0.0
s=1 and t=12: phi[1, 12] = 0.0
s=0 and t=13: phi[0, 13] = 0.0
s=1 and t=13: phi[1, 13] = 0.0
s=0 and t=14: phi[0, 14] = 0.0
s=1 and t=14: phi[1, 14] = 0.0
s=0 and t=15: phi